In [1]:
from os.path import abspath
from pyspark.sql import SparkSession

# set default location for [spark-warehouse]
warehouse_location = abspath('spark-warehouse')

In [2]:
spark = (
    SparkSession
    .builder
    .appName("Delta Lake")
    .config("spark.jars.packages", "org.apache.spark:spark-avro_2.12:3.1.1,io.delta:delta-core_2.12:1.0.0,org.apache.spark:spark-sql-kafka-0-10_2.12:3.1.1")
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")
    .config("spark.sql.debug.maxToStringFields","1000")
    .config("spark.kryoserializer.buffer.max","128m")
    .config("spark.driver.memory", "16G")
    .enableHiveSupport()
    .getOrCreate()
)
spark.sparkContext.setLogLevel("WARN")

:: loading settings :: url = jar:file:/usr/local/spark-3.1.2-bin-hadoop3.2/jars/ivy-2.4.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/jovyan/.ivy2/cache
The jars for the packages stored in: /home/jovyan/.ivy2/jars
org.apache.spark#spark-avro_2.12 added as a dependency
io.delta#delta-core_2.12 added as a dependency
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-813b821e-97ef-49a5-a2a9-8bad3798bcb8;1.0
	confs: [default]
	found org.apache.spark#spark-avro_2.12;3.1.1 in central
	found org.spark-project.spark#unused;1.0.0 in central
	found io.delta#delta-core_2.12;1.0.0 in central
	found org.antlr#antlr4;4.7 in central
	found org.antlr#antlr4-runtime;4.7 in central
	found org.antlr#antlr-runtime;3.5.2 in central
	found org.antlr#ST4;4.0.8 in central
	found org.abego.treelayout#org.abego.treelayout.core;1.0.3 in central
	found org.glassfish#javax.json;1.0.4 in central
	found com.ibm.icu#icu4j;58.2 in central
	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.1.1 in central
	found org.apache.spark#spark-token

In [3]:
conf = spark.sparkContext._jsc.hadoopConfiguration()
conf.set("fs.gs.project.id", "hering-datalake-prod")
conf.set("fs.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystem")
conf.set("fs.AbstractFileSystem.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFS")

---

In [5]:
!pip install google-cloud-speech
!pip install google-cloud-storage
!pip install google-cloud-bigquery

     |████████████████████████████████| 164 kB 7.5 MB/s            
     |████████████████████████████████| 46 kB 4.5 MB/s             
     |████████████████████████████████| 114 kB 55.6 MB/s            
     |████████████████████████████████| 1.0 MB 51.8 MB/s            
     |████████████████████████████████| 167 kB 56.2 MB/s            
     |████████████████████████████████| 211 kB 68.1 MB/s            
     |████████████████████████████████| 4.5 MB 61.0 MB/s            
     |████████████████████████████████| 155 kB 64.0 MB/s            
     |████████████████████████████████| 77 kB 5.6 MB/s             
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.18.1
    Uninstalling protobuf-3.18.1:
      Successfully uninstalled protobuf-3.18.1
     |████████████████████████████████| 106 kB 21.5 MB/s            
     |████████████████████████████████| 76 kB 5.3 MB/s             
     |████████████████████████████████| 211 kB 21.4 MB/s            
     |███████

In [6]:
from datetime import datetime
from itertools import count
from typing import List
from xmlrpc.client import Boolean
from dateutil import parser
import json
import pyspark.sql.functions as f
from pyspark.sql.types import StructField, StructType, StringType, DoubleType, BinaryType, IntegerType, TimestampType
from pyspark.sql import Column, DataFrame, SparkSession
from loguru import logger
from delta.tables import DeltaTable
import os
from google.cloud import storage
import decimal
import base64
from io import BytesIO
from zipfile import ZipFile

In [ ]:
#PATH LANDING ZONE
LANDING_PATH_VENDA = 'gs://hering-datalake-prod-southamerica-east1-bucket-landing-zone/bnsvher602/db_mobile/dbo/eih.bnsvher602.db_mobile.dbo.VENDA/'
LANDING_PATH_VENDA_ITEM = 'gs://hering-datalake-prod-southamerica-east1-bucket-landing-zone/bnsvher602/db_mobile/dbo/eih.bnsvher602.db_mobile.dbo.VENDA_ITEM/'
LANDING_PATH_FILIAL = 'gs://hering-datalake-prod-southamerica-east1-bucket-landing-zone/bnsvher602/db_mobile/dbo/eih.bnsvher602.db_mobile.dbo.FILIAL/'
LANDING_PATH_REDE_LOJA = 'gs://hering-datalake-prod-southamerica-east1-bucket-landing-zone/bnsvher602/db_mobile/dbo/eih.bnsvher602.db_mobile.dbo.REDE_LOJA/'
LANDING_PATH_EMPRESA = 'gs://hering-datalake-prod-southamerica-east1-bucket-landing-zone/bnsvher602/db_mobile/dbo/eih.bnsvher602.db_mobile.dbo.EMPRESA/'
LANDING_PATH_PRODUTO_BARRA = 'gs://hering-datalake-prod-southamerica-east1-bucket-landing-zone/bnsvher602/db_mobile/dbo/eih.bnsvher602.db_mobile.dbo.MB_PRODUTOS_BARRA/'
LANDING_PATH_VENDEDOR_LOJA = 'gs://hering-datalake-prod-southamerica-east1-bucket-landing-zone/bnsvher602/db_mobile/dbo/eih.bnsvher602.db_mobile.dbo.VENDEDOR_LOJA/'
LANDING_PATH_FILIAL_PONTO = 'gs://hering-datalake-prod-southamerica-east1-bucket-landing-zone/bnsvher602/db_mobile/dbo/eih.bnsvher602.db_mobile.dbo.FILIAL_PONTO/'
LANDING_PATH_SITE = 'gs://hering-datalake-prod-southamerica-east1-bucket-landing-zone/bnsvher602/db_mobile/dbo/eih.bnsvher602.db_mobile.dbo.SITE/'
LANDING_PATH_GERENTE_REGIONAL = 'gs://hering-datalake-prod-southamerica-east1-bucket-landing-zone/bnsvher602/db_mobile/dbo/eih.bnsvher602.db_mobile.dbo.GERENTE_REGIONAL/'
LANDING_PATH_CONSULTOR = 'gs://hering-datalake-prod-southamerica-east1-bucket-landing-zone/bnsvher602/db_mobile/dbo/eih.bnsvher602.db_mobile.dbo.CONSULTOR/'

LANDING_PATH_PRODUTOS =  'gs://hering-datalake-prod-southamerica-east1-bucket-landing-zone/bnsvher602/linx_db/dbo/eih.bnsvher602.linx_db.dbo.PRODUTOS/'
LANDING_PATH_PRODUTOS_BARRA_LINX = 'gs://hering-datalake-prod-southamerica-east1-bucket-landing-zone/bnsvher602/linx_db/dbo/eih.bnsvher602.linx_db.dbo.PRODUTOS_BARRA/'
LANDING_PATH_HR_WEB_CATALOGO_SKU =  'gs://hering-datalake-prod-southamerica-east1-bucket-landing-zone/bnsvher602/linx_db/dbo/eih.bnsvher602.linx_db.dbo.HR_WEB_CATALOGO_SKU/'
LANDING_PATH_HR_WEB_CATALOGO_FOTO =  'gs://hering-datalake-prod-southamerica-east1-bucket-landing-zone/bnsvher602/linx_db/dbo/eih.bnsvher602.linx_db.dbo.HR_WEB_CATALOGO_FOTO/'
LANDING_PATH_PRODUTO_CORES =  'gs://hering-datalake-prod-southamerica-east1-bucket-landing-zone/bnsvher602/linx_db/dbo/eih.bnsvher602.linx_db.dbo.PRODUTO_CORES/'

LANDING_PATH_BASE_CLIENTES =  'gs://hering-datalake-prod-southamerica-east1-bucket-landing-zone/bnsvher361/db_analise/crm/eih.bnsvher361.db_analise.crm.tb_base_clientes/'

#PATH BRONZE ZONE
BRONZE_PATH_VENDA = 'gs://hering-datalake-prod-southamerica-east1-bucket-bronze-zone/oto/bnsvher602/db_mobile/dbo/venda/'
BRONZE_PATH_VENDA_ITEM = 'gs://hering-datalake-prod-southamerica-east1-bucket-bronze-zone/oto/bnsvher602/db_mobile/dbo/venda_item/'
BRONZE_PATH_FILIAL = 'gs://hering-datalake-prod-southamerica-east1-bucket-bronze-zone/oto/bnsvher602/db_mobile/dbo/filial/'
BRONZE_PATH_REDE_LOJA = 'gs://hering-datalake-prod-southamerica-east1-bucket-bronze-zone/oto/bnsvher602/db_mobile/dbo/rede_loja/'
BRONZE_PATH_EMPRESA = 'gs://hering-datalake-prod-southamerica-east1-bucket-bronze-zone/oto/bnsvher602/db_mobile/dbo/empresa/'
BRONZE_PATH_PRODUTO_BARRA = 'gs://hering-datalake-prod-southamerica-east1-bucket-bronze-zone/oto/bnsvher602/db_mobile/dbo/mb_produtos_barra/'
BRONZE_PATH_VENDEDOR_LOJA = 'gs://hering-datalake-prod-southamerica-east1-bucket-bronze-zone/oto/bnsvher602/db_mobile/dbo/vendedor_loja/'
BRONZE_PATH_FILIAL_PONTO = 'gs://hering-datalake-prod-southamerica-east1-bucket-bronze-zone/oto/bnsvher602/db_mobile/dbo/filial_ponto/'
BRONZE_PATH_SITE = 'gs://hering-datalake-prod-southamerica-east1-bucket-bronze-zone/oto/bnsvher602/db_mobile/dbo/site/'
BRONZE_PATH_GERENTE_REGIONAL = 'gs://hering-datalake-prod-southamerica-east1-bucket-bronze-zone/oto/bnsvher602/db_mobile/dbo/gerente_regional/'
BRONZE_PATH_CONSULTOR = 'gs://hering-datalake-prod-southamerica-east1-bucket-bronze-zone/oto/bnsvher602/db_mobile/dbo/consultor/'

BRONZE_PATH_PRODUTOS =  'gs://hering-datalake-prod-southamerica-east1-bucket-bronze-zone/oto/bnsvher602/linx_db/dbo/produtos/'
BRONZE_PATH_PRODUTOS_BARRA_LINX = 'gs://hering-datalake-prod-southamerica-east1-bucket-bronze-zone/oto/bnsvher602/linx_db/dbo/produtos_barra/'
BRONZE_PATH_HR_WEB_CATALOGO_SKU =  'gs://hering-datalake-prod-southamerica-east1-bucket-bronze-zone/oto/bnsvher602/linx_db/dbo/hr_web_catalogo_sku/'
BRONZE_PATH_HR_WEB_CATALOGO_FOTO =  'gs://hering-datalake-prod-southamerica-east1-bucket-bronze-zone/oto/bnsvher602/linx_db/dbo/hr_web_catalogo_foto/'
BRONZE_PATH_PRODUTO_CORES =  'gs://hering-datalake-prod-southamerica-east1-bucket-bronze-zone/oto/bnsvher602/linx_db/dbo/produto_cores/'

BRONZE_PATH_BASE_CLIENTES =  'gs://hering-datalake-prod-southamerica-east1-bucket-bronze-zone/oto/bnsvher361/db_analise/crm/tb_base_clientes/'

#PATH SILVER ZONE
SILVER_PATH_VENDA = 'gs://hering-datalake-prod-southamerica-east1-bucket-silver-zone/oto/bnsvher602/db_mobile/dbo/venda/'
SILVER_PATH_VENDA_ITEM = 'gs://hering-datalake-prod-southamerica-east1-bucket-silver-zone/oto/bnsvher602/db_mobile/dbo/venda_item/'
SILVER_PATH_FILIAL = 'gs://hering-datalake-prod-southamerica-east1-bucket-silver-zone/oto/bnsvher602/db_mobile/dbo/filial/'
SILVER_PATH_REDE_LOJA = 'gs://hering-datalake-prod-southamerica-east1-bucket-silver-zone/oto/bnsvher602/db_mobile/dbo/rede_loja/'
SILVER_PATH_EMPRESA = 'gs://hering-datalake-prod-southamerica-east1-bucket-silver-zone/oto/bnsvher602/db_mobile/dbo/empresa/'
SILVER_PATH_PRODUTO_BARRA = 'gs://hering-datalake-prod-southamerica-east1-bucket-silver-zone/oto/bnsvher602/db_mobile/dbo/mb_produtos_barra/'
SILVER_PATH_VENDEDOR_LOJA = 'gs://hering-datalake-prod-southamerica-east1-bucket-silver-zone/oto/bnsvher602/db_mobile/dbo/vendedor_loja/'
SILVER_PATH_FILIAL_PONTO = 'gs://hering-datalake-prod-southamerica-east1-bucket-silver-zone/oto/bnsvher602/db_mobile/dbo/filial_ponto/'
SILVER_PATH_SITE = 'gs://hering-datalake-prod-southamerica-east1-bucket-silver-zone/oto/bnsvher602/db_mobile/dbo/site/'
SILVER_PATH_GERENTE_REGIONAL = 'gs://hering-datalake-prod-southamerica-east1-bucket-silver-zone/oto/bnsvher602/db_mobile/dbo/gerente_regional/'
SILVER_PATH_CONSULTOR = 'gs://hering-datalake-prod-southamerica-east1-bucket-silver-zone/oto/bnsvher602/db_mobile/dbo/consultor/'

SILVER_PATH_PRODUTOS =  'gs://hering-datalake-prod-southamerica-east1-bucket-silver-zone/oto/bnsvher602/linx_db/dbo/produtos/'
SILVER_PATH_PRODUTOS_BARRA_LINX = 'gs://hering-datalake-prod-southamerica-east1-bucket-silver-zone/oto/bnsvher602/linx_db/dbo/produtos_barra/'
SILVER_PATH_HR_WEB_CATALOGO_SKU =  'gs://hering-datalake-prod-southamerica-east1-bucket-silver-zone/oto/bnsvher602/linx_db/dbo/hr_web_catalogo_sku/'
SILVER_PATH_HR_WEB_CATALOGO_FOTO =  'gs://hering-datalake-prod-southamerica-east1-bucket-silver-zone/oto/bnsvher602/linx_db/dbo/hr_web_catalogo_foto/'
SILVER_PATH_PRODUTO_CORES =  'gs://hering-datalake-prod-southamerica-east1-bucket-silver-zone/oto/bnsvher602/linx_db/dbo/produto_cores/'

SILVER_PATH_BASE_CLIENTES =  'gs://hering-datalake-prod-southamerica-east1-bucket-silver-zone/oto/bnsvher361/db_analise/crm/tb_base_clientes/'
SILVER_PATH_CLIENTES_ABORDAVEIS = 'gs://hering-datalake-prod-southamerica-east1-bucket-silver-zone/oto/bnsvher361/db_analise/dbo/clientes_abordaveis_crm/'

#PATH GOLD ZONE
GOLD_PATH_OTO_DZARM = 'gs://hering-datalake-prod-southamerica-east1-bucket-gold-zone/oto/dzarm/'
GOLD_PATH_OTO_HERING_HMG = 'gs://hering-datalake-prod-southamerica-east1-bucket-gold-zone/oto/hering_hmg/'
GOLD_PATH_OTO_HERING_KIDS = 'gs://hering-datalake-prod-southamerica-east1-bucket-gold-zone/oto/hering_kids/'
GOLD_PATH_OTO_HERING_OUTLETS = 'gs://hering-datalake-prod-southamerica-east1-bucket-gold-zone/oto/hering_outlets/'

GOLD_PATH_STORES = 'gs://hering-datalake-prod-southamerica-east1-bucket-gold-zone/oto/view/stores/'

GOLD_PATH_OTO_CLIENTES_ABORDAVEIS = 'gs://hering-datalake-prod-southamerica-east1-bucket-gold-zone/oto/clientes_abordaveis/'


In [ ]:

def generete_symlink_manifest(spark:SparkSession, bucket:str) -> None:
        """
        Gera o arquivo symlink manifest para a tabela Delta do bucket de referencia
        """
        delta_table = DeltaTable.forPath(spark, bucket)
        delta_table.generate("symlink_format_manifest")


def path_exists(path, spark:SparkSession):
        """
        Dado um caminho verifica se existe ou não
        """
        sc = spark.sparkContext
        fs = sc._jvm.org.apache.hadoop.fs.FileSystem.get(
            sc._jvm.java.net.URI.create("gs://" + path.split("/")[2]),
            sc._jsc.hadoopConfiguration(),
        )
        return fs.exists(sc._jvm.org.apache.hadoop.fs.Path(path))


def last_date(delta_path:str, spark:SparkSession):
        """
        Retorna a data (timestamp_kafka) do último registro da tabela Delta
        """
        if not path_exists(delta_path + 'job_control/last_date', spark):
            try:
                df_temp = (
                    spark
                    .read
                    .format("delta")
                    .load(delta_path)
                )

                list_result = (
                    df_temp
                    .agg({"timestamp_kafka": "max"}).collect()
                )

                last_date = str(list_result[0].asDict()['max(timestamp_kafka)'])
                snapshot_flag = False
            except:
                snapshot_flag = True
                last_date = ('01-01-2021 12:00AM')

            df = spark.read.json(spark.sparkContext.parallelize([{'last_date': last_date}]))
            df.write.mode('overwrite').csv(delta_path + 'job_control/last_date', header = 'true')
        else:
            snapshot_flag = False
            df = spark.read.format("csv").load(delta_path + 'job_control/last_date', header = 'true')
            last_date = df.collect()[0].asDict()['last_date']

        logger.info('Last date: ' + last_date)

        return parser.parse(last_date),snapshot_flag


def update_last_date(delta_path:str, df:DataFrame, spark:SparkSession):
        """
        Atualiza a data (timestamp_kafka) do último registro da tabela Delta
        """
        list_result = (
            df
            .agg({"timestamp_kafka": "max"}).collect()
        )

        last_date = str(list_result[0].asDict()['max(timestamp_kafka)'])
        df = spark.read.json(spark.sparkContext.parallelize([{'last_date': last_date}]))
        df.write.mode('overwrite').csv(delta_path + 'job_control/last_date', header = 'true')

        return last_date

def get_operations_counts(df:DataFrame, spark:SparkSession):
        """
        Retorna o número de registros de cada operação kafka (__op)
        """
        op_count_list =(
            df
            .groupBy('__op')
            .count()
            .collect()
        )
        
        new_data_count = 0
        updating_data_count = 0
        deleting_data_count = 0

        if len(op_count_list) > 0:
            for i in op_count_list:
                dic_temp = i.asDict()
                if (dic_temp['__op'] == 'r'):
                    new_data_count = new_data_count + dic_temp['count']
                elif (dic_temp['__op'] != 'r'):
                    updating_data_count = updating_data_count + dic_temp['count']
                elif (dic_temp['__op'] == 'd'):
                    deleting_data_count = deleting_data_count + dic_temp['count']

        new_total_data_count = new_data_count + updating_data_count + deleting_data_count
        
        return new_total_data_count, new_data_count, updating_data_count, deleting_data_count


def get_data(spark:SparkSession, bucket:str, date_colunm_name:str, date_begin:datetime, source_format:str='delta', is_landing:bool=False, is_sink:bool=False):
        """
        Carrega Dataframe filtrando os dados do bucket de referência conforme 
        data de início enviada no parametro

        :param str date_begin: Data de início para filtro dos dados
        """
        if is_landing:

            if is_sink:
                df = (
                    spark
                    .read
                    .format(source_format)
                    .load(bucket)
                    .withColumn(date_colunm_name, (f.col(date_colunm_name)/1000).cast("timestamp"))
                    .where(
                        (f.col("year")>date_begin.year)|
                        (
                            (f.col("year")==date_begin.year)&
                            (
                            (f.col("month")>date_begin.month)|
                            (
                                (f.col("month")==date_begin.month)&
                                (
                                (f.col("day")>date_begin.day)|
                                (
                                    (f.col("day")==date_begin.day)&
                                    (f.col("hour")>=date_begin.hour)&
                                    (f.col(date_colunm_name)>date_begin)
                                )
                                )
                            )
                            )          
                        ) 
                    )
                )
            else:
                schema = StructType() \
                        .add("key",BinaryType(),True) \
                        .add("value",BinaryType(),True) \
                        .add("topic",StringType(),True) \
                        .add("partition",IntegerType(),True) \
                        .add("offset",StringType(),True) \
                        .add("timestamp",TimestampType(),True)
                df = (
                    spark
                    .read
                    .format(source_format)
                    .schema(schema)
                    .load(bucket)
                    .where(
                        (f.col("year")>date_begin.year)|
                        (
                            (f.col("year")==date_begin.year)&
                            (
                            (f.col("month")>date_begin.month)|
                            (
                                (f.col("month")==date_begin.month)&
                                (
                                (f.col("day")>date_begin.day)|
                                (
                                    (f.col("day")==date_begin.day)&
                                    (f.col("hour")>=date_begin.hour)&
                                    (f.col(date_colunm_name)>date_begin)
                                )
                                )
                            )
                            )          
                        ) 
                    )
                )
        else:
            df = (
                spark
                .read
                .format(source_format)
                .load(bucket)
                .where(
                    (f.col("year")>date_begin.year)|
                    (
                        (f.col("year")==date_begin.year)&
                        (
                        (f.col("month")>date_begin.month)|
                        (
                            (f.col("month")==date_begin.month)&
                            (
                            (f.col("day")>date_begin.day)|
                            (
                                (f.col("day")==date_begin.day)&
                                (f.col("hour")>=date_begin.hour)&
                                (f.col(date_colunm_name)>date_begin)
                            )
                            )
                        )
                        )          
                    ) 
                )
            )
        return df


def write_new_data(spark:SparkSession, df:DataFrame, bucket:str, destination_format:str='delta', partition_column:str=None):
        """
        Escreve no bucket de destino os dados novos, cuja a operação kafka é 'r'
        """
        if partition_column is not None:
            df_r = df.where((f.col('__op') == 'r'))
            (
                df_r
                .write
                .format(destination_format)
                .mode("append")
                .option("checkpointLocation", bucket + "checkpoint")
                .save(bucket)
            )
        else:
            (
                df
                .write
                .partitionBy(partition_column)
                .format(destination_format)
                .mode("append")
                .option("checkpointLocation", bucket + "checkpoint")
                .save(bucket)
            )
        
        return True


def update_new_data(spark:SparkSession, df:DataFrame, bucket:str, pk:list=['pk_single']):
        """
        Escreve no bucket de destino os dados novos, cuja a operação kafka é 'c' e
        atualiza os dados existentes, cuja a operação kafka é 'u' ou 'd'
        """
        table = DeltaTable.forPath(spark, bucket)
        df_table_u = df.where((f.col('__op') != 'r'))
        df_table_u_unique = df_table_u.orderBy('timestamp_kafka', ascending = False).dropDuplicates(pk)
        merge_string = 'destination.' + pk[0] + ' = updates.' + pk[0]
        (
            table.alias('destination')
            .merge(df_table_u_unique.alias('updates'), merge_string)
            .whenMatchedUpdateAll()
            .whenNotMatchedInsertAll()
            .execute()
        )
        
        return True


def delete_new_data(spark:SparkSession, bucket:str):
        """
        Deleta da tabela delta, no bucket de destino, os dados cuja a operação kafka é 'd'
        """
        table = DeltaTable.forPath(spark, bucket)
        table.delete(f.col('__op') == 'd')
        
        return True


def flatten_df(nested_df):
        """
        Retorna um dataframe com as colunas de um dataframe aninhado
        """
        flat_cols = [c[0] for c in nested_df.dtypes if c[1][:6] != 'struct']
        nested_cols = [c[0] for c in nested_df.dtypes if c[1][:6] == 'struct']

        flat_df = nested_df.select(flat_cols +
                                [f.col(nc+'.'+c)
                                    for nc in nested_cols
                                    for c in nested_df.select(nc+'.*').columns])
        return flat_df


def write_bronze_table(df:DataFrame, bucket:str):
        """
        Escreve no bucket de destino a tabela delta bronze
        """
        (
            df
            .write
            .partitionBy("year", "month", "day", "hour")
            .format("delta")
            .mode("append")
            .option("checkpointLocation", bucket + "checkpoint")
            .save(bucket)
        )

        return True

def get_schema(df:DataFrame):
    json_str = df.limit(1).toPandas()['fixedValue'][0]
    logger.info(json_str)
    json_fields = json.loads(json_str)['schema']['fields']
    struct_fields = []
    for field in json_fields:
        struct_fields.append(StructField(field['field'], StringType()))
        
    return StructType(struct_fields)

def decode_data_frame(df:DataFrame, schema:StructType):
    df_decoded = ( 
        df
        .withColumn('fixedValue', f.col("value").cast("string"))
        .select(f.get_json_object('fixedValue',"$.payload").alias('payload'), 'timestamp_kafka')
        .withColumnRenamed('timestamp', 'timestamp_kafka')
        .select(f.from_json(f.col('payload'), schema).alias("DF"), 'timestamp_kafka')
        .select("DF.*" , 'timestamp_kafka')
    )
    
    return df_decoded

def upload_blob_from_memory(bucket_name, contents, destination_blob_name):
    """Uploads a file to the bucket."""

    # The ID of your GCS bucket
    # bucket_name = "your-bucket-name"

    # The contents to upload to the file
    # contents = "these are my contents"

    # The ID of your GCS object
    # destination_blob_name = "storage-object-name"

    storage_client = storage.Client()
    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(destination_blob_name)

    blob.upload_from_string(contents)

def zip_file(nome_arquivo:str,contents):
    file_zip = BytesIO()
    with ZipFile(file_zip, 'w') as zipObj2:
        zipObj2.writestr(nome_arquivo,contents)
        zipObj2.close()
    return file_zip.getvalue()

def send_to_ftp(df:DataFrame, brand:str, table_name:str):

    temp_path = '/hering-datalake-prod-southamerica-east1-bucket-gold-zone/oto/temp/' + brand + '/'

    file_name = brand + '_' + table_name + '_' + datetime.today().strftime("%Y%m%d_%H%M%S")

    logger.info('Creating file...')
    df.coalesce(1).write.option("header",True).option("delimiter","|").csv('gs:/' + temp_path + file_name)

    storage_client = storage.Client()
    bucket = storage_client.get_bucket('hering-datalake-prod-southamerica-east1-bucket-gold-zone')

    blobs_list = list(bucket.list_blobs(prefix='oto/temp/' + brand + '/' + file_name + '/'))

    for file in blobs_list:
        try:
            if file.name.split('.')[-1] == 'csv':
                blob = file
        except:
            pass

    logger.info('Renaming file...')
    bucket.rename_blob(blob, 'oto/temp/' + brand + '/' + file_name + '.csv')

    nome_arquivo_csv = file_name + '.csv'
    nome_arquivo_zip = file_name + '.zip'
    bucket_name = 'hering-datalake-prod-southamerica-east1-bucket-gold-zone'
    blob = bucket.blob('oto/temp/' + brand + '/' + nome_arquivo_csv)
    contents = blob.download_as_string()

    contents_zip = zip_file(nome_arquivo_csv,contents)
    upload_blob_from_memory(bucket_name,contents_zip,'oto/temp/' + brand + '/' + nome_arquivo_zip)



        
    return None

@f.udf(returnType=DoubleType())
def big_java_decimal_to_double_python(big_decimal):
    bytes_val = base64.decodebytes(big_decimal.encode())
    bval = "".join("{0:08b}".format(c) for c in bytes_val)
    intval = int(bval, 2)

    return intval/(10**2)


def write_clientes_abordaveis(spark:SparkSession, df:DataFrame, bucket:str, brand:str):

    clientes_abordaveis_path = '/hering-datalake-prod-southamerica-east1-bucket-gold-zone/oto/clientes_abordaveis/' + brand + '/'
    file_name = brand + '_' + 'clientes_grupocontrole' + '_' + datetime.today().strftime("%Y%m%d_%H%M%S")

    logger.info('Creating file...')
    df.coalesce(1).write.option("header",True).option("delimiter","|").csv('gs:/' + clientes_abordaveis_path + file_name)

    storage_client = storage.Client()
    bucket = storage_client.get_bucket('hering-datalake-prod-southamerica-east1-bucket-gold-zone')

    blobs_list = list(bucket.list_blobs(prefix='oto/clientes_abordaveis/' + brand + '/' + file_name + '/'))

    for file in blobs_list:
        try:
            if file.name.split('.')[-1] == 'csv':
                blob = file
        except:
            pass

    logger.info('Renaming file...')
    bucket.rename_blob(blob, 'oto/clientes_abordaveis/' + brand + '/' + file_name + '.csv')

    nome_arquivo_csv = file_name + '.csv'
    nome_arquivo_zip = file_name + '.zip'
    bucket_name = 'hering-datalake-prod-southamerica-east1-bucket-gold-zone'
    blob = bucket.blob('oto/clientes_abordaveis/' + brand + '/' + nome_arquivo_csv)
    contents = blob.download_as_string()

    contents_zip = zip_file(nome_arquivo_csv,contents)
    upload_blob_from_memory(bucket_name,contents_zip,'oto/clientes_abordaveis/' + brand + '/' + nome_arquivo_zip)

In [ ]:
def get_last_date_jdbc(delta_path:str, spark:SparkSession):
        """
        Retorna a data do último registro da tabela Delta
        """
        if path_exists(delta_path + 'job_control/last_date', spark):
            first_data = False
            df = spark.read.format("csv").load(delta_path + 'job_control/last_date', header = 'true')
            last_date = df.collect()[0].asDict()['last_date']
        else:
            first_data = True
            last_date = ('01-01-1900 12:00AM')

        logger.info('Last date: ' + last_date)

        return parser.parse(last_date),first_data

In [ ]:
last_date, snapshot_flag = last_date(BRONZE_PATH_BASE_CLIENTES, spark)

In [ ]:
from pyspark.sql.avro.functions import from_avro, to_avro
import pyspark.sql.functions as f
from urllib.request import urlopen
import json
from pyspark.sql.types import StringType
from dateutil import parser

In [ ]:
def flatten_df(nested_df):
    flat_cols = [c[0] for c in nested_df.dtypes if c[1][:6] != 'struct']
    nested_cols = [c[0] for c in nested_df.dtypes if c[1][:6] == 'struct']

    flat_df = nested_df.select(flat_cols +
                               [f.col(nc+'.'+c)
                                for nc in nested_cols
                                for c in nested_df.select(nc+'.*').columns])
    return flat_df

In [ ]:
topic = 'bnsvher602.db_clientes.dbo.CRM_CLIENTES'
landing_path = 'gs://hering-datalake-prod-southamerica-east1-bucket-landing-zone/ecommerce/bnsvher602/db_clientes/dbo/crm_clientes/'
bronze_path = 'gs://hering-datalake-prod-southamerica-east1-bucket-bronze-zone/ecommerce/bnsvher602/db_clientes/dbo/crm_clientes/'
url = 'http://schema-registry-cp-schema-registry.ingestion.svc:8081/subjects/' + topic + '-value/versions/latest'

In [ ]:
url

In [ ]:
# Get Schema from schema registry
response = urlopen(url)
data_json = json.loads(response.read())

In [ ]:
last_date = parser.parse('21-03-2021 08:00AM')

In [ ]:
data_json

In [ ]:
# Reading last bronze data
try:
    df_bronze = (
        spark
        .read
        .format("delta")
        .load(bronze_path)
    )

    list_result = (
        df_bronze
        .agg({"timestamp_kafka": "max"}).collect()
    )
    
    last_date = list_result[0].asDict()['max(timestamp_kafka)']
    
except:
    last_date = parser.parse("01-01-2021 12:00AM")

In [ ]:
print(last_date)

In [ ]:
# Reading landing data
df_table = (
    spark
    .read
    .format("avro")
    .load(landing_path)
    .where((f.col("year")>=last_date.year)&(f.col("month")>=last_date.month)&(f.col("day")>=last_date.day))
    .where(f.col("timestamp")>last_date)
    .withColumnRenamed("timestamp", "timestamp_kafka")
)

In [ ]:
df_table.toPandas()

In [ ]:
from_avro_options= {"mode":"PERMISSIVE"}
df_temp = (
    df_table
    .withColumn('fixedValue', f.expr("substring(value, 6, length(value)-5)"))
    .select(from_avro("fixedValue", data_json['schema'], from_avro_options), 'offset', 'timestamp_kafka',"year", "month", "day", "hour")
)

In [ ]:
df_table_flat = flatten_df(df_temp)

In [ ]:
(
    df_table_flat
    .write
    .partitionBy("year", "month", "day", "hour")
    .format("delta")
    .mode("append")
    .option("checkpointLocation", bronze_path + "checkpoint")
    .save(bronze_path)
)

In [ ]:
##################################
### Verificar está funcionando ###
##################################

In [ ]:
count_bronze = (
    spark
    .read
    .format("delta")
    .load(bronze_path)
    .count()
)

print (count_bronze)

In [ ]:
landing_path_snapshot = 'gs://hering-datalake-prod-southamerica-east1-bucket-landing-zone/ecommerce/bnsvher602/linx_db/dbo/produtos_snapshot/'

count_landing = (
    spark
    .read
    .format("avro")
    .load(landing_path)
    .count()
)

count_landing_snap = (
    spark
    .read
    .format("avro")
    .load(landing_path_snapshot)
    .count()
)


print (count_landing + count_landing_snap)